# ADA - Project
## With Spark

## Getting started

In [1]:
from pyspark import SparkContext

In [2]:
sc = SparkContext("local", "ADA")

In [3]:
data = sc.parallelize(range(100))
data.sum()

4950

If you received 4950 as a result, your spark is working well :) Good job !

In [4]:
import pandas as pd

## Configuration

In [5]:
config = {
    "session":"progfun-002"
}

## Importing and parsing the data

### Importing Event Data

In [6]:
# Reading csv files: Create RDD () with one string entry per line in the file
rdd_problem_events = sc.textFile("../data/"+config['session']+"_Problem_Events_with_Info.csv")
rdd_video_events = sc.textFile("../data/"+config['session']+"_Video_Events.csv")
rdd_forum_events = sc.textFile("../data/"+config['session']+"_Forum_Events.csv")

# Prints the first line (header) along with indices for each table
print("--- PROBLEM_EVENTS ---")
for idx,field in enumerate(rdd_problem_events.first().split(",")): 
    print(idx,field, end=" / ")

print("\n\n--- VIDEO_EVENTS ---")
for idx,field in enumerate(rdd_video_events.first().split(",")): 
    print(idx,field, end=" / ")

print("\n\n--- FORUM_EVENTS ---")
for idx,field in enumerate(rdd_forum_events.first().split(",")): 
    print(idx,field, end=" / ")

--- PROBLEM_EVENTS ---
0 EventID / 1 ForumUserID / 2 MaximumSubmissions / 3 AccountUserID / 4 SubmissionNumber / 5 Grade / 6 TimeStamp / 7 DataPackageID / 8 ProblemID / 9 SoftCloseTime / 10 ProblemType / 11 HardCloseTime / 12 Platform / 13 OpenTime / 14 EventType / 15 Title / 16 SessionUserID / 17 UniqueProblemID / 18 UniqueUserID / 

--- VIDEO_EVENTS ---
0 EventID / 1 ForumUserID / 2 OldTime / 3 AccountUserID / 4 CurrentTime / 5 SeekType / 6 TimeStamp / 7 DataPackageID / 8 UniqueRowID / 9 TableName / 10 VideoID / 11 Platform / 12 NewSpeed / 13 EventSource / 14 EventType / 15 SessionUserID / 16 NewTime / 17 OldSpeed / 

--- FORUM_EVENTS ---
0 EventID / 1 ForumUserID / 2 PostID / 3 AccountUserID / 4 TimeStamp / 5 DataPackageID / 6 UniqueRowID / 7 TableName / 8 Platform / 9 EventSource / 10 PostType / 11 EventType / 12 JoinID / 13 SessionUserID / 

In [7]:
# Use the filter method to remove the first line
rdd_problem_events = rdd_problem_events.filter(lambda x: not x.startswith('EventID'))
rdd_video_events = rdd_video_events.filter(lambda x: not x.startswith('EventID'))
rdd_forum_events = rdd_forum_events.filter(lambda x: not x.startswith('EventID'))

# Prints first record for each table
print(rdd_problem_events.first(),"\n") 
print(rdd_video_events.first(),"\n") 
print(rdd_forum_events.first(),"\n")

3bcd1a54ed6ddb04b4a6fb2906110a01,None,0,None,7,None,1365344171,progfun-002,7,2147483640,Video,2147483640,Coursera,32400,Problem.Check,Lecture 1.2 - Elements of Programming (14:25),c4d4e5fcd2feba9f3234ee8d852dc7b22fbc07e4,f322944718b2ee0e53292118111533c7,21f13b3f6b50a83343b57d2f1d07dbdf 

db75adce6b87e7ab79242ea0af4b82d4,None,154.696,None,154.697,None,1372391638,progfun-002,00000078c0f0685cc50a25a8d5734a88,Video_Events,33,coursera,1.0,None,Video.Play,ef64fb7b096008f7eaf8441684afdf99af9af54a,None,1.0 

f3fdb52859b2511308aee554a573194e,None,17,4108315,1376254235,progfun-002,000006c12322ca29c7013dac42ef1a6a,Forum_Events,coursera,None,Thread,Forum.Thread.View,03b1fa287de5ef57d9c8482195b5167f,None 



In [8]:
# Function to parse the string entries from previous dataset
def parse_problems(x):
    data = x.split(',')
    return {
        'Grade':float(data[5]),
        'TimeStamp':int(data[6]),
        'Date':pd.to_datetime(int(data[6]),unit='s'),
        'ProblemID':int(data[8]),
        'ProblemType':data[10],
        'EventType':data[14].split('.')[0],
        'EventSubType':data[14].split('.')[1],
        'Title':data[15],
        'SessionUserID':data[16]
    }

def parse_videos(x):
    data = x.split(',')
    return {
        'TimeStamp':int(data[6]),
        'Date':pd.to_datetime(int(data[6]),unit='s'),
        'VideoID':int(data[10]),
        'EventType':data[14].split('.')[0],
        'EventSubType':data[14].split('.')[1],
        'SessionUserID':data[15]
    }

def parse_forums(x):
    data = x.split(',')
    return {
        'AccountUserID':data[3],
        'TimeStamp':int(data[4]),
        'Date':pd.to_datetime(int(data[4]),unit='s'),
        'EventType':data[11].split('.')[0],
        'EventSubType':data[11].split('.')[1]
    }

In [9]:
# Function to filter the string entries from previous dataset
def filter_problems(x):
    data = x.split(',')
    return (
        # Some problem event are quiz inside video, we decide to not consider these
        (data[10]=='Assignment')
        # We remove IDs of assignments that have been used by the teaching staff for testing the platform
        and (not data[8] in ['1','2','3','4','5'])
        # Discard assignments with no grade
        and (not data[5] in ['None'])
    )

def filter_videos(x):
    data = x.split(',')
    return (
        # We remove video that do not belong to the MOOC lectures or ar just Setup videos
        (data[10] not in ['9','12','11','10','13','2','29','25','21','27','23'])
        # Keeps only "Play" EnventSubStype for videos (makes difference between opening the page or starting the video)
        and (data[14].split('.')[1] in ['Play','Load'])
    )

def filter_forums(x):
    data = x.split(',')
    return (
        True
    )

In [10]:
# Use the map method to have more workable data
rdd_problem_events_parsed = rdd_problem_events.filter(filter_problems).map(parse_problems)
rdd_video_events_parsed = rdd_video_events.filter(filter_videos).map(parse_videos)
rdd_forum_events_parsed = rdd_forum_events.filter(filter_forums).map(parse_forums)

# Prints the count of elements along with the first element of each table
print(rdd_problem_events_parsed.count())
print(rdd_problem_events_parsed.first(),"\n") 
print(rdd_video_events_parsed.count()) 
print(rdd_video_events_parsed.first(),"\n") 
print(rdd_forum_events_parsed.count()) 
print(rdd_forum_events_parsed.first(),"\n") 

90325
{'Date': Timestamp('2013-04-17 17:47:58'), 'SessionUserID': 'd8f79efa32a560b8a46ea2b12d9bed97c9e39b4b', 'Grade': 9.32999992371, 'EventSubType': 'Check', 'Title': 'Functional Sets / Functional Sets', 'ProblemType': 'Assignment', 'EventType': 'Problem', 'ProblemID': 6, 'TimeStamp': 1366220878} 

1168226
{'Date': Timestamp('2013-06-28 03:53:58'), 'EventType': 'Video', 'EventSubType': 'Play', 'VideoID': 33, 'SessionUserID': 'ef64fb7b096008f7eaf8441684afdf99af9af54a', 'TimeStamp': 1372391638} 

297650
{'Date': Timestamp('2013-08-11 20:50:35'), 'EventType': 'Forum', 'AccountUserID': '4108315', 'TimeStamp': 1376254235, 'EventSubType': 'Thread'} 



In [11]:
# Handles bug with the forum events table having 'AccountUserID' instead of 'SessionUserID'
# Using the table progfun-002_User_Hash_Mapping
rdd_user_mapping = sc.textFile("../data/"+config['session']+"_User_Hash_Mapping.csv")
print(rdd_user_mapping.take(1)[0],"\n")

def f(x):
    x[1][1]['SessionUserID']=x[1][0] 
    return x[1][1]

rdd_forum_events_parsed = (rdd_user_mapping
    # removes header
    .filter(lambda x: not x.startswith("ForumUserID"))
    # maps to have the format (Key=AccountUserID,Value=SessionUserID)
    .map(lambda x:(x.split(",")[1],x.split(",")[6]))
    # join with rdd_forum_event to get format (map to have the (Key=AccountUserID,Value=(SessionUserID,EventObject)) format)
    .join(rdd_forum_events_parsed
        # map to have the (Key=AccountUserID,Value=EventObject) format
        .map(lambda x: (x['AccountUserID'],x))
    )
    # Use the function f to update EventObject with the joined SessionUserID
    .map(f)
)

print(rdd_forum_events_parsed.count())
print(rdd_forum_events_parsed.take(1)[0])

ForumUserID,AccountUserID,DataPackageID,UniqueRowID,TableName,Platform,SessionUserID 

297650
{'Date': Timestamp('2013-03-27 20:32:02'), 'EventType': 'Forum', 'AccountUserID': '1932792', 'EventSubType': 'ThreadSubscribe', 'SessionUserID': 'a97848d806f0d88cf80dd154845693af40cf559f', 'TimeStamp': 1364416322}


In [12]:
# Concatenantes all three table into one big table
rdd_events = (rdd_problem_events_parsed
    .union(rdd_video_events_parsed)
    .union(rdd_forum_events_parsed)
)
rdd_events.persist()
print(rdd_events.count())
print(rdd_events.take(1)[0])

1556201
{'Date': Timestamp('2013-04-17 17:47:58'), 'SessionUserID': 'd8f79efa32a560b8a46ea2b12d9bed97c9e39b4b', 'Grade': 9.32999992371, 'EventSubType': 'Check', 'Title': 'Functional Sets / Functional Sets', 'ProblemType': 'Assignment', 'EventType': 'Problem', 'TimeStamp': 1366220878, 'ProblemID': 6}


### Importing meta data
The goal is to have the table linking VideoIDs to corresponding ProblemIDs

In [13]:
assignments = sorted((rdd_problem_events_parsed
    .map(lambda x: (x['ProblemID'],x['Title']))
    .distinct()
    .collect()
),key=(lambda x: x[0]))

for item in assignments:
    print(item)

(6, 'Functional Sets / Functional Sets')
(7, 'Recursion / Recursion')
(12, 'Object-Oriented Sets / Object-Oriented Sets')
(14, 'Huffman Coding / Huffman Coding')
(17, 'Anagrams / Anagrams')
(20, 'Bloxorz / Bloxorz')


In [14]:
rdd_video_info = sc.textFile("../data/"+config['session']+"_Video_Info.csv")
videos = sorted((rdd_video_info
    .map(lambda x: x.split(','))
    .filter(lambda x: not x[1] in ['None', 'OpenTime'] )
    .map(lambda x: (int(x[7]),x[2]))
    .collect()
),key=(lambda x: x[0]))
          
for video in videos:
    print(video)

(2, 'Test Lecture 2')
(3, 'Lecture 1.2 - Elements of Programming (14:25)')
(4, 'Lecture 1.3 - Evaluation Strategies and Termination (4:22)')
(5, 'Lecture 1.4 - Conditionals and Value Definitions (8:49)')
(6, 'Lecture 1.5 - Example: square roots with Newton<squote/>s method (11:25)')
(7, 'Lecture 1.6 - Blocks and Lexical Scope (8:00)')
(8, 'Lecture 1.1 - Programming Paradigms (14:32)')
(9, 'Lecture 1.2')
(10, 'Lecture 1.3')
(11, 'Lecture 1.4')
(12, 'Lecture 1.5')
(13, 'Lecture 1.6')
(21, 'Tools Setup for Windows (10:37)')
(23, 'Tools Setup for Linux (12:24)')
(25, 'Tools Setup for Mac OS X (12:17)')
(27, 'Course Introduction (2:44)')
(29, 'Tutorial: Working on the Programming Assignments (8:47)')
(33, 'Lecture 1.7 - Tail Recursion (12:32)')
(35, 'Lecture 2.1 - Higher-Order Functions (10:18)')
(37, 'Lecture 2.2 - Currying (14:58)')
(39, 'Lecture 2.3 - Example: Finding Fixed Points (10:46)')
(41, 'Lecture 2.4 - Scala Syntax Summary (4:13)')
(43, 'Lecture 2.5 - Functions and Data (11:50)')

In [15]:
# Made by hand... so painful
LECTURES_PER_PROBLEM = {
    7: [3,4,5,6,7,8,33], # Lecture 1
    6: [35,37,39,41,43,47,49], # Lecture 2
    12: [51,53,75], # Lecture 3
    14: [71,81,85,79,87,77], # Lecture 4
    17: [109,105,115,107,103,113,111], # Lecture 6
    20: [123,117,125,121,127,119] # Lecture 7
}
LECTURES_PER_PROBLEM

{6: [35, 37, 39, 41, 43, 47, 49],
 7: [3, 4, 5, 6, 7, 8, 33],
 12: [51, 53, 75],
 14: [71, 81, 85, 79, 87, 77],
 17: [109, 105, 115, 107, 103, 113, 111],
 20: [123, 117, 125, 121, 127, 119]}

In [16]:
PROBLEM_PER_LECTURE = {}
for pb in LECTURES_PER_PROBLEM.keys():
    for lc in LECTURES_PER_PROBLEM[pb]:
        PROBLEM_PER_LECTURE[lc]=pb
PROBLEM_PER_LECTURE

{3: 7,
 4: 7,
 5: 7,
 6: 7,
 7: 7,
 8: 7,
 33: 7,
 35: 6,
 37: 6,
 39: 6,
 41: 6,
 43: 6,
 47: 6,
 49: 6,
 51: 12,
 53: 12,
 71: 14,
 75: 12,
 77: 14,
 79: 14,
 81: 14,
 85: 14,
 87: 14,
 103: 17,
 105: 17,
 107: 17,
 109: 17,
 111: 17,
 113: 17,
 115: 17,
 117: 20,
 119: 20,
 121: 20,
 123: 20,
 125: 20,
 127: 20}

## Working the data

### Grouping & Sorting

In [17]:
# uses the function groupByKey on our events with the key 'SessionStudentID'
rdd_events_by_students = (rdd_events
    .map(lambda x: (x['SessionUserID'],x))
    # Groups the list of event by student
    .groupByKey()
    # Sorts each student sequence by the timestamp
    .map(lambda x: (x[0],sorted(x[1], key=(lambda event: event['TimeStamp']))))
)
rdd_events_by_students.persist()
print("Number of students: %d" % rdd_events_by_students.count()) 

Number of students: 24626


In [18]:
# Function to have a friendly way to print the events
def eventToString(event):
    return {
        "Problem": lambda x: "(P "+str(event['ProblemID'])+")",
        "Video": lambda x: "(V "+str(event['VideoID'])+")",
        "Forum": lambda x: "(F)"
    }[event['EventType']](event)

In [19]:
TEST_STUDENTS={
    '6ea6949ca133acede360d3573f9d1168b3d70b51':'very good student',
    '5046ee71bc77983a0753d6e5ba98f5f1c685072e':'watched 4 videos',
    '3b305429f93de02637949578a5f9e23f13eb0726':'did two problems',
    '9625e050ef2326c12632c51aea7b5e49a20d6fc7':'good student',
    '865981d5b40a693bafbadae4b1df769be03a25c3':'watched 8 videos',
    '67cdae6073d1089b695e2a615a01187586ad7ba6':'normal student',
    'fb4c81b3df430d1f0fbb8d0ca3e470ac6bf92a2f':'the very best student',
    '295b3496278626b6d337812b1882b756336fd633':'whatdup with this guy?'
}

In [20]:
# Looks at the data for a few students
for studentID,events in rdd_events_by_students.filter(lambda x: x[0] in TEST_STUDENTS.keys()).collect():
    print('\n',studentID,':',TEST_STUDENTS[studentID])
    for event in events:
        print(eventToString(event), end=" ")
    print()


 6ea6949ca133acede360d3573f9d1168b3d70b51 : very good student
(V 8) (V 3) (V 3) (V 4) (V 5) (V 5) (V 6) (V 6) (V 6) (V 6) (V 6) (V 6) (V 7) (V 7) (V 7) (V 33) (V 33) (V 33) (P 7) (V 35) (V 35) (V 35) (V 35) (V 35) (V 35) (V 35) (V 35) (V 35) (V 35) (V 35) (V 35) (V 37) (V 37) (V 37) (V 37) (V 37) (V 37) (V 37) (V 37) (V 39) (V 39) (V 39) (V 41) (V 41) (V 43) (V 47) (V 49) (P 6) (P 6) (P 6) (V 51) (V 51) (V 51) (V 53) (V 53) (V 53) (V 75) (V 75) (V 75) (V 75) (P 12) (V 79) (V 77) (V 81) (V 83) (V 85) (V 87) (V 71) (P 14) (P 14) (V 73) (V 89) (V 91) (V 93) (V 95) (V 95) (V 97) (V 101) (P 14) (P 14) (P 14) (P 14) (P 14) (P 14) (P 14) (V 103) (V 105) (V 107) (V 109) (V 111) (V 113) (V 115) (F) (F) (F) (F) (F) (F) (F) (F) (P 17) (V 109) (V 111) (V 113) (V 107) (V 117) (V 119) (V 121) (V 123) (V 125) (V 127) (V 125) (F) (F) (P 20) (F) (F) (V 125) (V 125) (P 20) (F) 

 295b3496278626b6d337812b1882b756336fd633 : whatdup with this guy?
(V 8) (V 3) (V 3) (V 4) (V 5) (V 5) (V 6) (V 7) (V 33) (P 

### Extracting learning patterns
The goal is to extract what a students does between a failed attempt at an assignment and a successful attempt

In [21]:
# The goal is to extract the list corresponding to each Problem
def extractPatterns(events):
    patterns = {}
    for problemID in LECTURES_PER_PROBLEM.keys():
        patterns[problemID]=[]
    # for each problem measure first and last entry
    for event in events:
        if event['EventType']=='Problem':
            patterns[event['ProblemID']].append(event)
        if event['EventType']=='Video':
            if event['VideoID'] not in PROBLEM_PER_LECTURE.keys():
                print('WRONG VIDEO ID',event)
            else:
                patterns[PROBLEM_PER_LECTURE[event['VideoID']]].append(event)
    return patterns
    

In [22]:
# The goal is to concatenate small sub events into one big event
# For example Play/Pause/Play/Pause/Play/Pause on a video counts as only a Video event
def eventConcat(events):
    if len(events)<2:
        return events
    res=[events[0]]
    for event in events[1:-1]:
        if (
            event['EventType']!=res[-1]['EventType'] 
            or event.get('VideoID',None)!=res[-1].get('VideoID',None)
        ):
            res.append(event)
    res.append(events[-1])
    return res

def patternsConcat(patterns):
    res = {}
    for pattern in patterns.keys():
        res[pattern]=eventConcat(patterns[pattern])
    return res

In [23]:
rdd_events_by_students_extract = (rdd_events_by_students
    .map(lambda x: (x[0],extractPatterns(x[1])))
    .map(lambda x: (x[0],patternsConcat(x[1])))
)

rdd_events_by_students_extract.persist()

# Looks at the data for a student, now that it's sorted by TimeStamps
def displayStudentsPatterns(rdd,students):
    for studentID,patterns in rdd.filter(lambda x:x[0] in students.keys()).collect():
        print(studentID, students[studentID])
        for pb in sorted(patterns.keys()):
            print(pb,end='\t>>>  ')
            for event in patterns[pb]:
                print(eventToString(event), end=" ")
            print()
        print()

displayStudentsPatterns(rdd_events_by_students_extract,TEST_STUDENTS)

6ea6949ca133acede360d3573f9d1168b3d70b51 very good student
6	>>>  (V 35) (V 37) (V 39) (V 41) (V 43) (V 47) (V 49) (P 6) (P 6) 
7	>>>  (V 8) (V 3) (V 4) (V 5) (V 6) (V 7) (V 33) (P 7) 
12	>>>  (V 51) (V 53) (V 75) (P 12) 
14	>>>  (V 79) (V 77) (V 81) (V 85) (V 87) (V 71) (P 14) (P 14) 
17	>>>  (V 103) (V 105) (V 107) (V 109) (V 111) (V 113) (V 115) (P 17) (V 109) (V 111) (V 113) (V 107) 
20	>>>  (V 117) (V 119) (V 121) (V 123) (V 125) (V 127) (V 125) (P 20) (V 125) (P 20) 

295b3496278626b6d337812b1882b756336fd633 whatdup with this guy?
6	>>>  (V 35) (V 37) (V 39) (V 41) (V 43) (V 47) (V 49) (P 6) 
7	>>>  (V 8) (V 3) (V 4) (V 5) (V 6) (V 7) (V 33) (P 7) 
12	>>>  (V 51) (V 53) (V 75) (P 12) (P 12) 
14	>>>  (V 79) (V 77) (V 81) (V 85) (V 87) (V 71) (P 14) (P 14) 
17	>>>  (V 103) (V 105) (V 107) (V 109) (V 111) (V 113) (V 115) (P 17) (P 17) 
20	>>>  (V 121) (V 117) (V 119) (V 121) (V 123) (V 125) (P 20) (V 127) 

67cdae6073d1089b695e2a615a01187586ad7ba6 normal student
6	>>>  (V 35) (V 37)

In [24]:
def eventToLetter(event):
    return {
        "Problem": lambda x:"P",
        "Video": lambda x:"V",
        "Forum": lambda x:"F",
    }[event['EventType']](event)

def patternToSimpleString(pattern):
    return "".join([eventToLetter(event) for event in pattern])

# Uses the method flatMap on the students patterns table to have a table of all the patterns
rdd_patterns = (rdd_events_by_students_extract
    .flatMap(lambda x: [
        ((patternToSimpleString(x[1][pb]),pb),1) for pb in x[1].keys() 
    ])
    .filter(lambda x: 'P' in x[0][0])
    .reduceByKey(lambda a,b: a+b)
    .map(lambda x: (x[0][0],{'problem':x[0][1],'count':x[1]}))
    .groupByKey()
)

rdd_patterns.persist()

print(rdd_patterns.first())

('PVPVVVVVVVVV', <pyspark.resultiterable.ResultIterable object at 0x111de79e8>)


In [25]:
def sumCount(counts):
    return sum([i['count'] for i in counts])

# filters and shows the most common patterns 
for pattern in sorted(rdd_patterns.collect(), key=(lambda x: sumCount(x[1])), reverse=True)[:20]:
    print(sumCount(pattern[1]),'\t',pattern[0])
    for example in sorted(pattern[1],key=(lambda x: x['count']), reverse=True):
        print("\t",example)

5991 	 P
	 {'problem': 17, 'count': 1147}
	 {'problem': 7, 'count': 1104}
	 {'problem': 6, 'count': 1043}
	 {'problem': 20, 'count': 988}
	 {'problem': 12, 'count': 903}
	 {'problem': 14, 'count': 806}
5388 	 VVVVVVVP
	 {'problem': 7, 'count': 2674}
	 {'problem': 6, 'count': 1428}
	 {'problem': 17, 'count': 731}
	 {'problem': 20, 'count': 376}
	 {'problem': 14, 'count': 131}
	 {'problem': 12, 'count': 48}
4628 	 PP
	 {'problem': 14, 'count': 935}
	 {'problem': 6, 'count': 901}
	 {'problem': 12, 'count': 901}
	 {'problem': 17, 'count': 650}
	 {'problem': 20, 'count': 639}
	 {'problem': 7, 'count': 602}
3508 	 VVVVVVVPP
	 {'problem': 7, 'count': 1254}
	 {'problem': 6, 'count': 1174}
	 {'problem': 20, 'count': 454}
	 {'problem': 17, 'count': 384}
	 {'problem': 14, 'count': 197}
	 {'problem': 12, 'count': 45}
1896 	 VVVVVVPP
	 {'problem': 14, 'count': 1225}
	 {'problem': 20, 'count': 428}
	 {'problem': 12, 'count': 118}
	 {'problem': 7, 'count': 47}
	 {'problem': 6, 'count': 44}
	 {'proble

In [26]:
rdd_onestep = (rdd_patterns
    .filter(lambda x: len(x[0])>1)
    .flatMap(lambda x: [((int(grade['from']),x[0][1:2]),1) for grade in x[1]])
    .reduceByKey(lambda a,b:a+b)
    .map(lambda x: (x[0][0],(x[0][1],x[1])))
    .groupByKey()
)

print('Grade','\t','P','\t','V','\t','F')
print()
for item in rdd_onestep.take(10):
    print(item[0], end='\t')
    counts={}
    for event,count in list(item[1]):
        counts[event]=count
    print('{0:.2f}'.format(counts['P']/sum(counts.values())),end='\t')
    print('{0:.2f}'.format(counts['V']/sum(counts.values())),end='\t')
    print('{0:.2f}'.format(counts['F']/sum(counts.values())),end='\t')
    print()

Grade 	 P 	 V 	 F



Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.runJob.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 51.0 failed 1 times, most recent failure: Lost task 0.0 in stage 51.0 (TID 305, localhost): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/Users/louisfaucon/Documents/spark-2.0.1-bin-hadoop2.7/python/lib/pyspark.zip/pyspark/worker.py", line 172, in main
    process()
  File "/Users/louisfaucon/Documents/spark-2.0.1-bin-hadoop2.7/python/lib/pyspark.zip/pyspark/worker.py", line 167, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/Users/louisfaucon/Documents/spark-2.0.1-bin-hadoop2.7/python/pyspark/rdd.py", line 2371, in pipeline_func
    return func(split, prev_func(split, iterator))
  File "/Users/louisfaucon/Documents/spark-2.0.1-bin-hadoop2.7/python/pyspark/rdd.py", line 2371, in pipeline_func
    return func(split, prev_func(spli